# Step 1: Loading train, test


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVR
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import keras_tuner as kt

# Metrics calculation
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return {"MAE": mae, "RMSE": rmse, "MAPE": mape}

# XGBoost Training
def train_xgboost(X_train, y_train, X_test, y_test, dates_train, dates_test):
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=50)
    model.fit(X_train, y_train)
    
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    metrics_train = calculate_metrics(y_train, y_pred_train)
    metrics_test = calculate_metrics(y_test, y_pred_test)
    return metrics_train, metrics_test, model

# Random Forest Training
def train_random_forest(X_train, y_train, X_test, y_test, dates_train, dates_test):
    model = RandomForestRegressor(n_estimators=50)
    model.fit(X_train, y_train)

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    metrics_train = calculate_metrics(y_train, y_pred_train)
    metrics_test = calculate_metrics(y_test, y_pred_test)
    return metrics_train, metrics_test, model

# LightGBM Training
def train_lightgbm(X_train, y_train, X_test, y_test, dates_train, dates_test):
    model = lgb.LGBMRegressor()
    model.fit(X_train, y_train)

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    metrics_train = calculate_metrics(y_train, y_pred_train)
    metrics_test = calculate_metrics(y_test, y_pred_test)
    return metrics_train, metrics_test, model

# SVR Training
def train_svr(X_train, y_train, X_test, y_test, dates_train, dates_test):
    params = {'C': [0.1, 1, 0.01], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']}
    model = SVR()
    grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='neg_mean_squared_error', cv=3)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    y_pred_train = best_model.predict(X_train)
    y_pred_test = best_model.predict(X_test)
    
    metrics_train = calculate_metrics(y_train, y_pred_train)
    metrics_test = calculate_metrics(y_test, y_pred_test)
    return metrics_train, metrics_test, best_model

# Deep Learning Model Builder
def build_deep_learning_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Fine-Tuned Deep Learning Training
def train_fine_tuned_deep_learning(X_train, y_train, X_test, y_test, dates_train, dates_test):
    tuner = kt.BayesianOptimization(
        build_deep_learning_model,
        objective='val_loss',
        max_trials=20,
        directory='kt_dir',
        project_name='fine_tuned_deep_learning'
    )
    tuner.search(X_train, y_train, epochs=50, validation_split=0.2, verbose=0)
    
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(X_train, y_train, epochs=150, batch_size=32, validation_split=0.2, verbose=0,
                   callbacks=[EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)])
    
    y_pred_train = best_model.predict(X_train).flatten()
    y_pred_test = best_model.predict(X_test).flatten()
    
    metrics_train = calculate_metrics(y_train, y_pred_train)
    metrics_test = calculate_metrics(y_test, y_pred_test)
    return metrics_train, metrics_test, best_model

# Model Training Function
def train_models(X_train, y_train, X_test, y_test, dates_train, dates_test, i):
    results_train = {"Series_Number": i, "Models": {}}
    results_test = {"Series_Number": i, "Models": {}}

    print("Training XGBoost...")
    xgb_train_metrics, xgb_test_metrics, _ = train_xgboost(X_train, y_train, X_test, y_test, dates_train, dates_test)
    results_train["Models"]["XGBoost"] = xgb_train_metrics
    results_test["Models"]["XGBoost"] = xgb_test_metrics

    print("Training Random Forest...")
    rf_train_metrics, rf_test_metrics, _ = train_random_forest(X_train, y_train, X_test, y_test, dates_train, dates_test)
    results_train["Models"]["Random Forest"] = rf_train_metrics
    results_test["Models"]["Random Forest"] = rf_test_metrics

    print("Training LightGBM...")
    lgbm_train_metrics, lgbm_test_metrics, _ = train_lightgbm(X_train, y_train, X_test, y_test, dates_train, dates_test)
    results_train["Models"]["LightGBM"] = lgbm_train_metrics
    results_test["Models"]["LightGBM"] = lgbm_test_metrics

    print("Training SVR...")
    svr_train_metrics, svr_test_metrics, _ = train_svr(X_train, y_train, X_test, y_test, dates_train, dates_test)
    results_train["Models"]["SVR"] = svr_train_metrics
    results_test["Models"]["SVR"] = svr_test_metrics

    print("Training Fine-Tuned Deep Learning...")
    ft_dl_train_metrics, ft_dl_test_metrics, _ = train_fine_tuned_deep_learning(X_train, y_train, X_test, y_test, dates_train, dates_test)
    results_train["Models"]["Fine-Tuned Deep Learning"] = ft_dl_train_metrics
    results_test["Models"]["Fine-Tuned Deep Learning"] = ft_dl_test_metrics

    # Convert to DataFrame for better visualization
    df_results_train = pd.DataFrame.from_dict(results_train["Models"], orient='index').reset_index()
    df_results_test = pd.DataFrame.from_dict(results_test["Models"], orient='index').reset_index()

    df_results_train["Series_Number"] = i
    df_results_test["Series_Number"] = i
    print("Results for training data:")
    print(df_results_train)
    print("Results for testing data:")
    print(df_results_test)
    return df_results_train, df_results_test

C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## MAIN

In [2]:
import pandas as pd

start_series = 25  # Specify the starting series number
end_series = 40   # Specify the ending series number

# Lists to collect all results for concatenation
all_results_train = []  
all_results_test = []

# Loop through the defined range
for i in range(start_series, end_series + 1):

    train_path = f"C:\\Users\\Siwar\\Research_project\\First approach\\Datasets\\3_Data_ready_for_modelling\\Train\\serie_{i}train.csv"
    test_path = f"C:\\Users\\Siwar\\Research_project\\First approach\\Datasets\\3_Data_ready_for_modelling\\Test\\serie_{i}test.csv"
    
    # Load datasets
    train_data = pd.read_csv(train_path, index_col="Date")
    test_data = pd.read_csv(test_path, index_col="Date")
    
    # Prepare X and y
    X_train = train_data.drop(columns=['y'])
    X_test = test_data.drop(columns=['y'])
    y_train = train_data['y']
    y_test = test_data['y']
    
    # Train models and save results
    results_train, results_test = train_models(X_train, y_train, X_test, y_test, train_data.index, test_data.index, i)
    
    # Collect all results for concatenation
    all_results_train.append(results_train)
    all_results_test.append(results_test)

# Concatenate all the results into a single DataFrame
final_results_train = pd.concat(all_results_train, ignore_index=True)
final_results_test = pd.concat(all_results_test, ignore_index=True)

# Save the concatenated results 
train_results_path = f"C:\\Users\\Siwar\\Research_project\\First approach\\Datasets\\4_Results_metrics\\all_results_train_series_{start_series}_to_{end_series}.csv"
test_results_path = f"C:\\Users\\Siwar\\Research_project\\First approach\\Datasets\\4_Results_metrics\\all_results_test_series_{start_series}_to_{end_series}.csv"

final_results_train.to_csv(train_results_path, index=False)
final_results_test.to_csv(test_results_path, index=False)

print(f"Results saved for series {start_series} to {end_series}.")


Training XGBoost...


C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Training Random Forest...


C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Training LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000849 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5785
[LightGBM] [Info] Number of data points in the train set: 400, number of used features: 45
[LightGBM] [Info] Start training from score 448.817500
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

...

C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mea

Training LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001057 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5821
[LightGBM] [Info] Number of data points in the train set: 400, number of used features: 45
[LightGBM] [Info] Start training from score 437.387500
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Training Fine-Tuned Deep Learning...
Reloading Tuner from kt_dir\fine_tuned_deep_learning\tuner0.json
1/1 [==============================] - 0s 31ms/step
Results for training data:
                      index        MAE       RMSE      MAPE  Series_Number
0                   XGBoost   0.390227   0.513285  0.091269             40
1             Random Forest   2.853650   3.970065  0.641111             40
2                  LightGBM   2.030398   2.949099  0.450096             40
3                       SVR  17.090907  23.544536  3.843500             40
4  Fine-Tuned Deep Learning  17.259726  23.549571  3.872901             40
Results for testing data:
                      index         MAE        RMSE       MAPE  Series_Number
0                   XGBoost  124.330832  125.772411  24.127039             40
1             Random Forest   93.561429   95.510987  18.045645             40
2                  LightGBM   99.769565  101.342623  19.262529             40
3                       SVR   1

C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Siwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
